# ASoC/AARDVARC UDP readout example
This example notebooks show how a user can read out the data from the ASoC or AARDVARC eval cards connected to the Nexys video FPGA card.

### Naludaq Version
*Min Version*: `0.26.4`

In [ ]:
# Print Naludaq version
import naludaq

print(f"Naludaq version: {naludaq.__version__}")

### Compatible Boards
+ `AARDVARCv3`
+ `HDSOCv1_evalr2`
+ `ASOCv3`
+ `AODSv2`
+ `TRBHM`
+ `AODSOC_AODS`
+ `AODSOC_ASOC`


## imports and variables

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time

# Imports for board creation and identification
from naluconfigs import get_available_models
from naludaq.board import Board, startup_board

# Registers modules handles communication with FPGA and ASIC registers.
from naludaq.communication import ControlRegisters, DigitalRegisters, AnalogRegisters

# Controllers controls one aspect of the board, board controllers start/stop acquisitons, readout controllers set the readout parameters.
from naludaq.controllers import get_board_controller, get_readout_controller

# Imports for data acquisition.
import pathlib

In [ ]:
from logging import getLogger, Formatter, StreamHandler, INFO, DEBUG


def setup_logger(level=INFO):
    """Setup a basic logger.

    Logging to the stream formatted for easy visual readout.

    Args:
        level: Logging level, ex. logging.INFO

    Returns:
        logger object.
    """
    logger = getLogger()
    handler = StreamHandler()
    handler.setFormatter(
        Formatter("%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s")
    )
    logger.addHandler(handler)
    logger.setLevel(DEBUG)
    uart = getLogger("naludaq.board.connections._UART")
    uart.setLevel(DEBUG)

    return logger


try:
    logger.debug("logger already setup")  # noqa
except NameError:
    logger = setup_logger(DEBUG)

## Create board object

In [ ]:
# List all possible board model names:
print("\n".join([x for x in get_available_models()]))

In [ ]:
model = "asocv3"  # Board model all in lowercase

In [ ]:
BOARD = Board(model)

## Startup a backend server

The backend server is a separate process that handles the communication with the FPGA board.
The server can be run as a separate command line program which can be downloaded from here: [support.naluscientific.com](https://support.naluscientific.com)

Start a local server, with the same lifeitme/scope as the python program.
Set the working directory for the server. This is where the data will be saved.

In [ ]:
output_dir = pathlib.Path().cwd() / "output"
output_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
BOARD.start_server(output_dir=str(output_dir))

In [ ]:
# Full details of the backend API can be found at:
print(f"http://{BOARD._context.address[0]}:{BOARD._context.address[1]}/api")

Connect the board object to the backend server and have the backend server establish a connection to the board.

In [ ]:
BOARD.connect_udp(
    board_addr=("192.168.22.40", 4660),
    receiver_addr=("192.168.22.129", 4660),
    # backend_addr=('127.0.0.1', 7878),  # Optional, if you want to connect to an external backend, only if you don't run `BOARD.start_server`
)

In [ ]:
with BOARD:
    startup_board(BOARD)

It is possible to set the IP address and port of the board using the ControlRegisters module. The IP address and port of the board can be set using the following commands:
```python
ControlRegisters.write('eth_dest_ip', 'ip address')
ControlRegisters.write('eth_dest_port', 4661)
````


In [ ]:
BC = get_board_controller(BOARD)

In [ ]:
with BOARD:
    print(BC.read_firmware_version())

In [ ]:
CR = ControlRegisters(BOARD)
AR = AnalogRegisters(BOARD)
DR = DigitalRegisters(BOARD)

In [ ]:
# Test writing and reading from ASIC registers.
with BOARD:
    DR.write("chipid", 0b1010101)  # 85
    print(DR.read("chipid"))

In [ ]:
def enable_serial(enable=True):
    CR.write("iomode0", not enable)
    CR.write("iomode1", enable)


with BOARD:
    enable_serial(
        False
    )  # Enable serial connection between ASIC and FPGA, DON'T ENABLE UNLESS YOU KNOW WHAT YOU ARE DOING

# Readout event

Since the board is using UDP between the hardware and the backend, we then use TCP between the backend and the notebook.  
The AcquisitionManager is used to control acquisitions.

#### Setup an Acquisition to store data into:

In [ ]:
from naludaq.backend import AcquisitionManager

AM = AcquisitionManager(BOARD)

In [ ]:
# Create an acquisition
ACQUISITION = AM.create()
print("Created acquisition with name:", ACQUISITION.name)

In [ ]:
# List available acquisitions
print("Available acquisitions:")
for x in [acq.name for acq in AM.list()]:
    print(f"    {x}")

In [ ]:
# Set an acquisition as output for data
ACQUISITION.set_output()
print("Current output acquisition:", AM.current_acquisition.name)

Once up an Acquisition is created and set as output, the data can be read from the board to the Acquisition

### Setup readout parameters

Once the board trigger it continues to sample for `write after trigger` cycles.  
Then it will move the starting point for the readout `lookback` windows.  
Lastly it will readout `windows` samples from the lookback point.  

This allow the number of samples before and after the trigger point to be set in increments of windows.
Window is the atom unit, one window is 64 samples wide.

In the example we use 8 windows (512 samples), write after trig 4 (256 sp), and lookback 8 (512 sp). Which means 256 samples before and after the trigger.

| Important to note that the boards trigger on a window by window basis and NOT per sample, which can cause edges to not line up in overlapping events. | | --- |


In [ ]:
with BOARD:
    get_readout_controller(BOARD).set_read_window(
        windows=8,
        lookback=8,
        write_after_trig=4,
    )
    get_readout_controller(BOARD).set_readout_channels(
        [0, 1, 2, 4]
    )  # Can be left out to read all channels

### Simplest readout
The simplest way is to set the board up to listen for X number of seconds.

The data will be stored in the output directory.

To open the data see, [opening acquisitions](opening_acquisitions.ipynb)

> the try - finally statement will catch the KeyboardInterup and allow the operation to be cancelled with `ctrl+c`

In [ ]:
with BOARD:
    get_board_controller(BOARD).start_readout(
        "imm"
    )  # Start readout in immediate mode, the board will trigger itself.
    try:
        time.sleep(3)
    finally:
        get_board_controller(BOARD).stop_readout()

### Simplest externally Triggered readout

The simplest triggered readout is to trigger the board externally and readout the data. 

The board will wait for a trigger signal on trigger input port and then readout the data.

It's possible to start and stop the readout manually without having a wait statement by adding a condition that waits for user input to continue.

This section will capture the triggers during the time interval then stop.

In [ ]:
with BOARD:
    # Make sure the readout paramters are set correctly

    get_board_controller(BOARD).start_readout(
        "ext"  # Start readout in external trigger mode, the board will trigger on software triggers or trig_in input.
    )
    try:
        time.sleep(3)  # Capture any triggers in the time interval.
    finally:
        get_board_controller(BOARD).stop_readout()

### Externally Triggered readout waiting for X events instead of time

In [ ]:
from naludaq.tools.data_collector import get_data_collector

In [ ]:
COLLECTOR = get_data_collector(BOARD)

In [ ]:
# Don't forget you can always ask for help
help(COLLECTOR)

In [ ]:
# Setting up the readout settings is a bit different with the data collector
COLLECTOR.channels = [0, 1, 2, 3]  # Can be left out to read all channels
COLLECTOR.forced = False  # IF True the board will readout based on memory address instead of trigger position. DON'T USE.
COLLECTOR.set_external_trigger()
COLLECTOR.set_window(
    windows=8,
    lookback=8,
    write_after_trig=4,
)

In [ ]:
### Create a data capture pipeline
num_captures = 10
num_evt_to_throw_away = 10  # warmup captures


def validator(x):
    """A function that returns True if the data is valid, False otherwise."""
    return True


def inner(x):
    """When calling with enumerated data the argument is a tuple with the index in the block and the data."""
    idx = x[0]
    _ = x[1]  # The data
    print(f"Capturing event {idx + 1}/{num_captures}")


pipeline = COLLECTOR.iter_inf(attempts=10)
pipeline = pipeline.for_each(lambda _: print("Do something here with data"))
pipeline = pipeline.filter(validator, exclusion_limit=10)
pipeline = (
    pipeline.enumerate().for_each(inner).unenumerate()
)  # Example of how an event can be enumerated.
pipeline = pipeline.skip(num_evt_to_throw_away)
with BOARD:
    pipeline.take(num_captures).collect()

### Analog signal triggering

The boards have capability to trigger on in the input signal.

Levels can be set for each channel, and the trigger can be set to trigger on a rising or falling edge.

To determine the optimal levels, use the threshold scan function in NaluScope.

> Different boards have more or less granularity in the triggering circuit, check documentation.

In [ ]:
from naludaq.controller.trigger import get_trigger_controller

TC = get_trigger_controller(BOARD)

In [ ]:
with BOARD:
    # Set the triggers
    TC.trigger_values = [
        1000,
        1500,
        2000,
        2500,
    ]  # Set the trigger values as represented by the thresholdscan.
    TC.set_trigger_edge(rising=True)  # Set the trigger edge to rising or falling.

    get_board_controller(BOARD).start_readout(
        "self"
    )  # Start readout in immediate mode, the board will trigger itself.
    try:
        time.sleep(3)  # Capture any triggers in the time interval.
    finally:
        get_board_controller(BOARD).stop_readout()

### Pedestals

Pedestals are an important part of the workflow. To generate and store the pedestals with the acquisition, follow the steps below:

In [ ]:
from naludaq.tools.pedestals.generators import get_pedestals_generator

PG = get_pedestals_generator(BOARD)

In [ ]:
with BOARD:
    PG.generate_pedestals()

In [ ]:
# Associate the pedestals with the current acquisition:
AM.current_acquisition.pedestals = BOARD.pedestals

In [ ]:
# The pedestals can then be accessed as an attribute of the acquisition:
AM.current_acquisition.pedestals